## Using kugupu to calculate molecular coupling networks

This notebook demonstrates how to calculate molecular coupling between fragments, inspect the results and save and load these results.

In [11]:
import MDAnalysis as mda
import kugupu as kgp
import numpy as np
import yaehmop

Firstly we create an `MDAnalysis.Universe` object.  This Universe will need to have bonds and atom names.

In [12]:
u = mda.Universe('./out.pdb.bz2', '../kugupu/data/last20.dcd')

This system has 46,500 atoms in 250 different fragments.

In [41]:
print(u.atoms.n_atoms, u.atoms.n_fragments)

46500 250


Our dynamics simulation has 20 frames of results.

In [43]:
print(u.trajectory.n_frames)

20


The coupling matrix is calculated using the `kgp.coupling_matrix` function.

Here we are calculating the coupling matrix for fragments in the Universe `u` where
- coupling is calculated between fragments with a closest approach of less than 5.0 Angstrom
- coupling is calculated between the LUMO upwards
- one state per fragment is considered
- the analysis stops at frame 5
- with a stride of 2 frames over the trajectory 

In [21]:
res = kgp.coupling_matrix(u, nn_cutoff=5.0, state='lumo', degeneracy=1, stop=5, step=2)

2019-04-12T11:58:43.214539+0100 INFO Processing 3 frames
2019-04-12T11:58:43.515784+0100 INFO Processing frame 1 of 3
2019-04-12T11:58:43.818020+0100 INFO Finding dimers within 5.0, passed 250 fragments
2019-04-12T11:58:45.888178+0100 INFO Found 2244 dimers
100%|██████████| 2244/2244 [01:22<00:00, 30.15it/s]
2019-04-12T12:00:08.354719+0100 INFO Processing frame 2 of 3
2019-04-12T12:00:08.720357+0100 INFO Finding dimers within 5.0, passed 250 fragments
2019-04-12T12:00:10.760621+0100 INFO Found 2134 dimers
100%|██████████| 2134/2134 [01:22<00:00, 28.44it/s]
2019-04-12T12:01:33.292721+0100 INFO Processing frame 3 of 3
2019-04-12T12:01:33.696083+0100 INFO Finding dimers within 5.0, passed 250 fragments
2019-04-12T12:01:35.871967+0100 INFO Found 2133 dimers
100%|██████████| 2133/2133 [01:21<00:00, 26.32it/s]
2019-04-12T12:02:56.919020+0100 INFO Done!


The `res` object is a namedtuple which contains all the data necessary to perform further analysis.
This object has various attributes which will not be briefly explained.

The `.frames` attribute records which frames from the trajectory were analysed.
This is useful to later compare positions of fragments with the coupling values.

In [27]:
print(res.frames)

[0 2 4]


The `.degeneracy` attribute stores how many degenerate states were considered for each fragment.
This value will not change over time, so this array has shape `nfragments`.

In this example only a single state per fragment was considered. 

In [28]:
print(res.degeneracy)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


The `.H_frag` attribute contains the molecular coupling values, stored inside a 3d numpy array.
The first dimension is along the number of frames (quasi time axis),
while the other two move along fragments in the system.

For example `res.H_frag[0, 1, 12]` gives the coupling (in eV) between the 2nd and 13th fragments in the first frame.

In [33]:
print(res.H_frag.shape)

print(res.H_frag[0, 1, 12])

(3, 250, 250)
2.9479195706142796e-05


Producing these results is often a time consuming part of the analysis,
therefore it is wise to save them to a file so you can come back to them later!

This can be done using the `kugupu.save_results` function, which will save the results to a hdf5 (compressed) format.
These results can later be retrieved using the `kugupu.load_results` function.

In [35]:
kgp.save_results('myresults.hdf5', res)

These results can then be retrieved again using the `kugupu.load_results` function:

In [37]:
kgp.load_results('./myresults.hdf5')

KugupuResults(frames=array([0, 2, 4]), H_frag=array([[[-1.10273229e+01,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.12064372e+01,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -1.03646964e+01, ...,
          0.00000000e+00,  5.03681664e-04,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.03393511e+01,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  5.03681664e-04, ...,
          0.00000000e+00, -1.02979008e+01,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -1.05113036e+01]],

       [[-1.09286240e+01,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.07579111e+01,  0.00000000e+0